In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
n_input = 28 * 28
n_noise = 128

n_hidden = 256 #은닉층 노드 갯수
total_epoch = 100
batch_size = 100
learning_rate = 0.0002 

In [0]:
X = tf.placeholder(tf.float32, [None, n_input])
# Z는 생성망에 들어 갈 입력 데이터 (정규분포를 따르는 128개의 데이터)
Z = tf.placeholder(tf.float32, [None, n_noise])

In [0]:
# 생성자 변수 선언/ 생성자 신경망
# W/b 설정

In [0]:
# 생성자신경망(입력층->은닉층)
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
#ㅅ애성자 신경망(은닉층-> 출력층)
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [0]:
# 이미지 판별망
# 입력층 -> 은닉층으로 
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

#은닉층에서 출력층으로
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

#### 노이즈 데이터 발생을 위한 노이즈 생성함수
#### def get_noise(데이터갯수, 노이즈개수)

In [0]:
def get_noise(batch_size, n_noise):
  return np.random.normal(size=(batch_size, n_noise))

In [0]:
#생성자와 신경망을 직접 구성

In [0]:
def generator(noise_z):
  hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
  
  return output

In [0]:
# 판별자 신경망 구성

In [0]:
def discriminator(inputs):
  hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
  
  return output

In [0]:
# 생성자 신경망
G = generator(Z)

# 판별자 신경망
D_fake = discriminator(G)

# 진짜 이미지
D_real = discriminator(X)

### 비용함수// 최적화함수

In [0]:
# 경찰관의 성능 -> 가능한 높은 값이 좋다
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_fake))

# 위조지폐범의 성능 -> 가능한 높은 값이 좋다
loss_G = tf.reduce_mean(tf.log(D_fake))

In [0]:
# 사용하는 변수 리스트 화
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [0]:
### 오차 최소화가 아니라,
### GAN은 loss_D와 loss_G를 최대화시키는 것이 목표다!

In [0]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list = D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list = G_var_list)

In [0]:
# 학습시키기
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0,0

In [0]:
for epoch in range(200):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size) #판별망에 들어가는 데이터
    noise = get_noise(batch_size, n_noise) # 생성망에 들어가는 입력 데이터
    
    _, loss_val_D = sess.run([train_D, loss_D],
                            feed_dict = {X:batch_xs, Z:noise})
    
    _, loss_val_G = sess.run([train_G, loss_G],
                            feed_dict = {Z:noise})
    
  print("Epoch : {}, 판별망의 성능 : {:.4f}, 생성망의 성능 : {:.4f}".format(epoch, loss_val_D, loss_val_G))
    
print('최적화 완료!')

Epoch : 0, 판별망의 성능 : -0.6975, 생성망의 성능 : -1.8215
Epoch : 1, 판별망의 성능 : -0.6901, 생성망의 성능 : -2.1227
Epoch : 2, 판별망의 성능 : -0.6321, 생성망의 성능 : -1.7466
Epoch : 3, 판별망의 성능 : -0.6811, 생성망의 성능 : -2.1621
Epoch : 4, 판별망의 성능 : -0.6933, 생성망의 성능 : -2.2028
Epoch : 5, 판별망의 성능 : -0.6633, 생성망의 성능 : -2.0326
Epoch : 6, 판별망의 성능 : -0.5787, 생성망의 성능 : -2.0630
Epoch : 7, 판별망의 성능 : -0.5636, 생성망의 성능 : -2.0061
Epoch : 8, 판별망의 성능 : -0.5964, 생성망의 성능 : -1.9891
Epoch : 9, 판별망의 성능 : -0.6026, 생성망의 성능 : -2.0102
Epoch : 10, 판별망의 성능 : -0.5991, 생성망의 성능 : -2.5252
Epoch : 11, 판별망의 성능 : -0.5929, 생성망의 성능 : -2.1081
Epoch : 12, 판별망의 성능 : -0.6600, 생성망의 성능 : -2.0551
Epoch : 13, 판별망의 성능 : -0.6489, 생성망의 성능 : -2.1327
Epoch : 14, 판별망의 성능 : -0.5807, 생성망의 성능 : -2.4601
Epoch : 15, 판별망의 성능 : -0.6669, 생성망의 성능 : -1.9916
Epoch : 16, 판별망의 성능 : -0.6976, 생성망의 성능 : -2.1985
Epoch : 17, 판별망의 성능 : -0.5897, 생성망의 성능 : -2.1028
Epoch : 18, 판별망의 성능 : -0.5287, 생성망의 성능 : -2.5484
Epoch : 19, 판별망의 성능 : -0.5993, 생성망의 성능 : -2.5266
Epoch : 20, 판별망의 성능 : -0.5514,

In [0]:
! mkdir samples

In [0]:
if epoch ==  0 or (epoch+1) % 10 == 0:
  sample_size = 10
  noise = get_noise(sample_size, n_noise)
  samples = sess.run(G, feed_dict = {Z: noise})

In [0]:
fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

for i in range(sample_size):
  ax[i].set_axis_off()
  ax[i].imshow(np.reshape(samples[i], (28, 28)))
  
  
plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight' )
plt.close(fig)
  
print('최적화 완료!')

최적화 완료!
